In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# reads in data from csv
file_path = "./resources/park_data_raw.csv"
raw_df = pd.read_csv(file_path)

In [3]:
# Brings back entries that are NOT home films
df = raw_df[raw_df.title != "@home"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1373 entries, 7 to 1379
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    1370 non-null   object 
 1   park                     1373 non-null   object 
 2   date                     1135 non-null   object 
 3   phone                    1372 non-null   object 
 4   rating                   1356 non-null   object 
 5   cc                       1223 non-null   object 
 6   location.latitude        1336 non-null   float64
 7   location.longitude       1336 non-null   float64
 8   location.human_address2  1169 non-null   object 
 9   datayear                 1373 non-null   int64  
 10  day                      1135 non-null   object 
 11  park_address             647 non-null    object 
 12  address                  1371 non-null   object 
 13  city                     1366 non-null   object 
 14  state                    1366

In [4]:
# Drops rows missing data in necessary columns
df2 = df.dropna(how="any", subset=["community", "location.latitude", "location.longitude", "title", "address", "date", "rating"])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1096 entries, 7 to 1379
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    1096 non-null   object 
 1   park                     1096 non-null   object 
 2   date                     1096 non-null   object 
 3   phone                    1096 non-null   object 
 4   rating                   1096 non-null   object 
 5   cc                       1007 non-null   object 
 6   location.latitude        1096 non-null   float64
 7   location.longitude       1096 non-null   float64
 8   location.human_address2  893 non-null    object 
 9   datayear                 1096 non-null   int64  
 10  day                      1096 non-null   object 
 11  park_address             624 non-null    object 
 12  address                  1096 non-null   object 
 13  city                     1089 non-null   object 
 14  state                    1089

In [5]:
# drops unneccesary columns
df3 = df2.drop(columns=["phone", "zip", "park_address", "class", "type", "osm_type", "lat",
                        "long", "state", "city", "display_name", "location.human_address2"])
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1096 entries, 7 to 1379
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1096 non-null   object 
 1   park                1096 non-null   object 
 2   date                1096 non-null   object 
 3   rating              1096 non-null   object 
 4   cc                  1007 non-null   object 
 5   location.latitude   1096 non-null   float64
 6   location.longitude  1096 non-null   float64
 7   datayear            1096 non-null   int64  
 8   day                 1096 non-null   object 
 9   address             1096 non-null   object 
 10  community           1096 non-null   object 
 11  geocode_address     1096 non-null   object 
 12  communitiy          1096 non-null   object 
 13  community_match     1096 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 128.4+ KB


In [6]:
# Cleans cells
df4 = df3.replace({
    # Inconsistent Format
    "Y": "Yes", 
    "N": "No",
    None: "No",
    # Title Correction
    "Wizard of Oz": "The Wizard of Oz",
    "ET": "E.T. The Extra-Terrestrial",
    "X-Man: Days of Future Past": "X-Men: Days of Future Past",
    "Monster University": "Monsters University",
    # Title Spacing
    "E. T. : The Extra-Terrestrial": "E.T. The Extra-Terrestrial",
    "E. T. the Extra-Terrestrial": "E.T. The Extra-Terrestrial",
    "The Wiz ": "The Wiz",
    "Gravity ": "Gravity",
    "Fat Albert ": "Fat Albert",
    "Ferris Bueller's Day Off ": "Ferris Bueller's Day Off",
    "The Smurfs 2 ": "The Smurfs 2",
    "Teenage Mutant Ninja Turtles (1990) ": "Teenage Mutant Ninja Turtles (1990)",
    "Chicago ": "Chicago",
    "No Good Deed ": "No Good Deed"
    })

In [7]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1096 entries, 7 to 1379
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1096 non-null   object 
 1   park                1096 non-null   object 
 2   date                1096 non-null   object 
 3   rating              1096 non-null   object 
 4   cc                  1096 non-null   object 
 5   location.latitude   1096 non-null   float64
 6   location.longitude  1096 non-null   float64
 7   datayear            1096 non-null   int64  
 8   day                 1096 non-null   object 
 9   address             1096 non-null   object 
 10  community           1096 non-null   object 
 11  geocode_address     1096 non-null   object 
 12  communitiy          1096 non-null   object 
 13  community_match     1096 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 128.4+ KB


In [8]:
df4.title.unique()

array(['Lego Movie', 'The Smurfs 2', 'Delivery Man',
       'A League of Their Own', 'Frozen', 'Monsters University',
       'The Nut Job', 'Free Birds', 'Hairspray', 'The Wizard of Oz',
       'The Muppet Movie', 'Annie',
       'Willy Wonka and the Chocolate Factory', 'Brave',
       'The Perfect Game', 'Newsies', 'Saving Mr. Banks', 'Ironman 3',
       'Gremlins', "It's a Mad, Mad, Mad, Mad World", 'Gravity',
       'Avengers', 'The Blues Brothers', 'The Lego Movie',
       'Despicable Me 2', 'Superman', 'The Family That Preys',
       'E.T. The Extra-Terrestrial', 'To Kill a Mockingbird', '42',
       'Drumline', 'Ride Along', "Singin' in the Rain",
       'Willy Wonka & the Chocolate Factory', 'Back to the Future',
       'Man of Steel', 'The Help', 'Rudy', 'Star Trek: Into Darkness',
       'Dirty Dancing', 'The Bachelor and the Bobby Soxer',
       'Strangers on a Train', 'The Thin Man', 'Fireproof ',
       'Instructions Not Included', 'Mandela: Long Walk to Freedom',
       'J

In [9]:
# Here is where we get the month
df5 = df4.date.str.split(pat="-", expand=True)
df5

,0,1,2
7,2014,08,08T03:00:00Z
8,2014,07,19T03:00:00Z
9,2014,06,14T04:00:00Z
10,2014,06,08T02:45:00Z
11,2014,06,21T03:00:00Z
...,...,...,...
1375,2019,08,24T05:00:00Z
1376,2019,08,24T05:00:00Z
1377,2019,08,26T05:00:00Z
1378,2019,09,07T05:00:00Z


In [10]:
# renaming columns
df5 = df5.rename(columns={
    0: "trash1",
    1: "datamonth",
    2: "trash2"
})
df6 = df5.drop(columns=["trash1", "trash2"])
# merge dataframes
df4 = df4.join(df6)

In [11]:
# last column drop
df4 = df4.drop(columns=["communitiy", "community_match"])
df4

,title,park,date,rating,cc,location.latitude,location.longitude,datayear,day,address,community,geocode_address,datamonth
7,Lego Movie,Austin-Foster Playlot Park,2014-08-08T03:00:00Z,PG,No,41.975941,-87.778908,2014,Fri,"6020 W Foster Ave Chicago, IL 60630",JEFFERSON PARK,"6020 W Foster Ave Chicago, IL 60630",08
8,The Smurfs 2,Brooks Park,2014-07-19T03:00:00Z,PG,No,42.010281,-87.806686,2014,Sat,"7100 N Harlem Ave Chicago, IL 60631",EDISON PARK,"7100 N Harlem Ave Chicago, IL 60631",07
9,Delivery Man,Dunham Park,2014-06-14T04:00:00Z,PG-13,No,41.965981,-87.781975,2014,Sat,4638 N. Melvina Ave. Chicago IL 60630,PORTAGE PARK,4638 N. Melvina Ave. Chicago IL 60630,06
10,A League of Their Own,Edgebrook Park,2014-06-08T02:45:00Z,PG,No,41.999466,-87.762024,2014,Sun,6525 N. Hiawatha Ave. Chicago IL 60646,FOREST GLEN,6525 N. Hiawatha Ave. Chicago IL 60646,06
11,Lego Movie,Olympia Park,2014-06-21T03:00:00Z,PG,No,41.999702,-87.814987,2014,Sat,6566 N. Avondale Ave. Chicago IL 60631,EDISON PARK,6566 N. Avondale Ave. Chicago IL 60631,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,Aquaman,Euclid Park,2019-08-24T05:00:00Z,PG-13,Yes,41.716042,-87.637148,2019,Sat,9800 S. Parnell Ave.,WASHINGTON HEIGHTS,"9800 S. Parnell Ave., Chicago, IL",08
1376,The Wizard of Oz,Oz Park,2019-08-24T05:00:00Z,G,Yes,41.918901,-87.647178,2019,Sat,2021 N. Burling St.,LINCOLN PARK,"2021 N. Burling St., Chicago, IL",08
1377,The Wiz,South Shore Cultural Center,2019-08-26T05:00:00Z,G,Yes,41.766603,-87.566324,2019,Mon,7059 S. South Shore Drive,SOUTH SHORE,"7059 S. South Shore Drive, Chicago, IL",08
1378,After School Cartoons,Buttercup Playlot Park,2019-09-07T05:00:00Z,NR,No,41.971669,-87.654739,2019,Sat,4901 N. Sheridan Rd.,UPTOWN,"4901 N. Sheridan Rd., Chicago, IL",09


In [12]:
df4.to_csv("resources/clean_park_data.csv", index=False)